In [ ]:
!pip install openai
!pip install requests

In [ ]:
youtube_url = "https://www.youtube.com/watch?v=9WWvLj-NqEE"

In [ ]:
def extract_youtube_id(youtube_url):
    """Extracts the YouTube video ID from a URL."""
    if 'v=' in youtube_url:
        return youtube_url.split('v=')[1].split('&')[0]
    return None

video_id = extract_youtube_id(youtube_url)
print(f"Extracted video ID: {video_id}")

In [ ]:
import os

# Only prompt for API key if not already set in environment
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    api_key = getpass('Enter your API key: ')
    api_token = getpass('Enter your youtube-transcript.io API token: ')

    os.environ["OPENAI_API_KEY"] = api_key
    os.environ["YOUTUBE_TRANSCRIPT_IO"] = api_token
else:
    api_key = os.environ["OPENAI_API_KEY"]
    api_token = os.environ["YOUTUBE_TRANSCRIPT_IO"]

os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

In [ ]:
import requests

url = "https://www.youtube-transcript.io/api/transcripts"
headers = {
    "Authorization": f"Basic {api_token}",
    "Content-Type": "application/json"
}
data = {"ids": [video_id]}

response = requests.post(url, headers=headers, json=data)

In [ ]:
data = response.json()
# If the response is a list of dicts, use indexing:
item = data[0]

# extract the transcript segments
transcript_segments = item['tracks'][0]['transcript']

# If transcript_segments is a list of segments with a 'text' field:
full_transcript = " ".join(segment['text'] for segment in transcript_segments)


In [ ]:
from openai import OpenAI
import os
from IPython.display import Markdown, display

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_API_BASE"]
)

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a world-class, award-winning blog writer. "
                "Your task is to write a summary blog post based only on the provided transcript. "
                "Format the output in clear Markdown. Use headings, bullet points, bolding, and code where appropriate. "
                "Produce only a concise summary—no full transcript, and no details outside the transcript."
            )
        },
        {
            "role": "user",
            "content": f"Write a Markdown summary blog post based on the following transcript:\n\n{full_transcript}"
        }
    ]
)

markdown_output = response.choices[0].message.content
display(Markdown(markdown_output))